In [11]:
import inseq
from inseq.commands.attribute_context.attribute_context import attribute_context_with_model, AttributeContextArgs
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import json
import torch
import numpy as np
import pandas as pd

In [12]:
double_quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True)
model_double_quant = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct", quantization_config=double_quant_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
inseq_model = inseq.load_model(
    model_double_quant,
    "saliency",
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct") 

The model is loaded in 4bit mode. The device cannot be changed after loading the model.


In [19]:
df.head()

,Unnamed: 0,query_x,answer_x,options_x,sum_supports,supports
0,0,What event did Juan Rossell participate in?,1996 summer olympics,"['1996 summer olympics', 'olympic games', 'spo...","The 2004 Summer Olympic Games, held in Athens...","The 2004 Summer Olympic Games, officially know..."
1,1,What languages did John Osteen speak or write?,english,"['english', 'greek', 'koine greek', 'nahuatl',...",Christianity is a monotheistic religion based...,A Christian (or ) is a person who follows or a...
2,2,What is the parent taxon of Australosuchus?,crocodilia,"['animal', 'area', 'crocodile', 'crocodilia', ...",Mekosuchinae was a subfamily of crocodiles th...,Mekosuchinae was a subfamily of crocodiles fro...
3,3,What is the'record_label method man' associate...,loud records,"['1995', '1996', 'album', 'english', 'epic', '...","Wu-Tang Clan's debut album, ""Enter the Wu-Tan...",Enter the Wu-Tang (36 Chambers) is the debut s...
4,4,In which administrative territorial entity is ...,raipur district,"['bangladesh', 'bhopal', 'bhutan', 'canada', '...","Chhattisgarh, a state in central India, is ri...",A province is almost always an administrative ...


In [3]:
inseq_model = inseq.load_model(
    "microsoft/Phi-3-mini-4k-instruct",
    "saliency",
)

ds = load_dataset('saracandu/hotpotQA_nli', split= "train")

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct") 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [60]:
ds = load_dataset('saracandu/hotpotQA_nli', split= "train")

In [90]:
df = pd.read_csv('hotpot_bridge.csv')

df.head()

,Unnamed: 0,question,correct,alternative,type,level_y,selected_passages
0,0,The Oberoi family is part of a hotel company t...,Delhi,Mumbai,bridge,medium,The Oberoi family is an Indian family that is ...
1,1,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,President Abraham Lincoln,bridge,hard,"Allison Beth ""Allie"" Goertz (born March 2, 199..."
2,2,What nationality was James Henry Miller's wife?,American,British,bridge,medium,James Henry Miller (25 January 1915 – 22 Octob...
3,3,Cadmium Chloride is slightly soluble in this c...,alcohol,Sodium Chloride,bridge,medium,Cadmium chloride is a white crystalline compou...
4,4,Which genus of moth in the world's seventh-lar...,Crambidae,Lepidoptera,bridge,hard,"India, officially the Republic of India (""Bhār..."


In [77]:
# FUNZIONI AUSILIARIE VARIE

def top_k_positions(input_context_scores, k):
    # Convert the input array to a numpy array if it's not already
    input_context_scores = np.array(input_context_scores)
    
    # Use argsort to get the indices of the top 8 highest elements
    top_k_indices = np.argsort(input_context_scores)[-k:]
    
    # Reverse to get the indices in descending order of their values
    top_k_indices = top_k_indices[::-1]
    
    return top_k_indices.tolist()

# -------------------------------------------------------------------------

def estrai_frasi_con_token(frasipositioni, posizioni_token):
    # Creiamo un set per le posizioni dei token per una ricerca più veloce
    set_posizioni_token = set(posizioni_token)
    
    # Lista per memorizzare le frasi di contesto
    frasi_contesto = []
    
    for frase, inizio, fine in frasipositioni:
        # Verifica se c'è almeno un token rilevante nell'intervallo [inizio, fine]
        if any(token in set_posizioni_token for token in range(inizio, fine + 1)):
            frasi_contesto.append(frase)
    
    return frasi_contesto

# -------------------------------------------------------------------------

def find_positions(vector, target="<0x0A>"):
    positions = []
    for i, element in enumerate(vector):
        if element == target:
            positions.append(i)   
    return positions

# -------------------------------------------------------------------------

def map_relative_positions(abs_positions, text):
    temp = []
    start = 0
    for i in range(len(abs_positions)):
        if i != 0: 
            start = abs_positions[i-1] + 1
        temp.append([split_text(text)[i], start, abs_positions[i]])

    return temp

# -------------------------------------------------------------------------

def select_passages(text_passage, question, p, out, tokens):
    
    num_of_sets = len(out.cci_scores)
    
    top_scores = []
    for i in range(num_of_sets):
        top_scores.extend(top_k_positions(out.cci_scores[i].input_context_scores, p))
    positions = list(set(top_scores))

    return '\n'.join(estrai_frasi_con_token(map_relative_positions(find_positions(tokens), text_passage), positions))
 
# -------------------------------------------------------------------------

def split_text(text):
    # Split the text on ". " and return the resulting list
    return text.split("\n ")

# -------------------------------------------------------------------------

def invoke_pecore(text_passage, question, p):
    pecore_args = AttributeContextArgs(
        model_name_or_path="microsoft/Phi-3-mini-4k-instruct",
        attribution_method="saliency",
        attributed_fn="contrast_prob_diff",
        context_sensitivity_metric="kl_divergence",
        context_sensitivity_std_threshold=1,
        context_sensitivity_topk = find_top_p(text_passage, p),
        attribution_std_threshold=None,
        attribution_topk=None,
        input_current_text=question, 
        input_context_text=text_passage,
        contextless_input_current_text="""<|system|>
    You are a helpful assistant that provide concise and accurate answers.<|end|>
    <|user|>
    {current}<|end|>
    <|assistant|>""",
        input_template="""<|system|>
    You are a helpful assistant that provide concise and accurate answers.<|end|>
    <|user|>
    {context}
    
    {current}<|end|>
    <|assistant|>""",
        contextless_output_current_text="""{current}""",
        output_template="{current}",
        special_tokens_to_keep=['<|system|>', '<|end|>', '<|assistant|>', '<|user|>'],
        decoder_input_output_separator="""
    """,
        save_path=None,
        viz_path=None,
        show_viz=False,
        generation_kwargs={'max_new_tokens': 50},
    )

    out = attribute_context_with_model(pecore_args, inseq_model) 
    return out

# -------------------------------------------------------------------------

def find_top_p(text_passage, p):
    
    tokens = tokenizer.tokenize(text_passage)
    num_topk_tokens = int(p/100 * len(tokens)) 

    return num_topk_tokens

In [88]:
def run(question, passage, p):
    
    tokens = tokenizer.tokenize(passage)
    out = invoke_pecore(passage, question, p)
    return select_passages(passage, question, p, out, tokens)

In [62]:
test = invoke_pecore(ds['question'][1], ds['passages'][1], 5, )

In [63]:
print(test.cci_scores[0].input_context_scores)

[0.07125589996576309, 0.0755460113286972, 0.053215391933918, 0.08854875713586807, 0.08749596029520035, 0.03789963945746422, 0.031167736276984215, 0.08002915978431702, 0.06544961780309677, 0.0791502371430397, 0.03600878641009331, 0.0393093079328537, 0.05297643691301346, 0.09505551308393478, 0.036038074642419815, 0.04878504201769829, 0.029188279062509537, 0.07565844058990479, 0.04307213053107262]


In [89]:
run(df['question'][1], df['selected_passages'][1], 5)

'Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician.\nGoertz is known for her satirical songs based on various pop culture topics.\nSubjects of her songs have included the film "The Room", the character Milhouse from the television show "The Simpsons", and the game Dungeons & Dragons.\nHer style has been compared to that of Bo Burnham.\nIn December 2015, Goertz released a concept album based on the Adult Swim series "Rick and Morty", "Sad Dance Songs", with the album\'s cover emulating the animation and logo of the series.\nShe is co-host of Everything\'s Coming Up Podcast, a Simpsons-focused podcast along with Julia Prescott. Milhouse Mussolini van Houten is a fictional character featured in the animated television series "The Simpsons", voiced by Pamela Hayden, and created by Matt Groening who named the character after President Richard Nixon\'s middle name.'

In [84]:
num_of_sets = len(test.cci_scores)
    
top_scores = []
for i in range(num_of_sets):
    top_scores.extend(top_k_positions(test.cci_scores[i].input_context_scores, 5))
positions = list(set(top_scores))

tokens = tokenizer.tokenize(df['support'][1])
print(find_positions(tokens))
# estrai_frasi_con_token(map_relative_positions(find_positions(tokens), df['support'][1]), positions)

[]


In [93]:
tokens = tokenizer.tokenize(df['selected_passages'][1])
select_passages2(df['selected_passages'][1], df['question'][1], 5, test, tokens)

['Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician.',
 'Goertz is known for her satirical songs based on various pop culture topics.']

In [94]:
run(df['question'][1], df['selected_passages'][1], 5)

'Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician.\nGoertz is known for her satirical songs based on various pop culture topics.\nSubjects of her songs have included the film "The Room", the character Milhouse from the television show "The Simpsons", and the game Dungeons & Dragons.\nHer style has been compared to that of Bo Burnham.\nIn December 2015, Goertz released a concept album based on the Adult Swim series "Rick and Morty", "Sad Dance Songs", with the album\'s cover emulating the animation and logo of the series.\nShe is co-host of Everything\'s Coming Up Podcast, a Simpsons-focused podcast along with Julia Prescott. Milhouse Mussolini van Houten is a fictional character featured in the animated television series "The Simpsons", voiced by Pamela Hayden, and created by Matt Groening who named the character after President Richard Nixon\'s middle name.'

In [85]:
split_text(df['support'][1])

['Allison Beth "Allie" Goertz (born March 2, 1991) is an American musician.  Goertz is known for her satirical songs based on various pop culture topics.  Her videos are posted on YouTube under the name of Cossbysweater.  Subjects of her songs have included the film "The Room", the character Milhouse from the television show "The Simpsons", and the game Dungeons & Dragons.  Her style has been compared to that of Bo Burnham.  In December 2015, Goertz released a concept album based on the Adult Swim series "Rick and Morty", "Sad Dance Songs", with the album\'s cover emulating the animation and logo of the series.  The album was made possible through Kickstarter.  She is co-host of Everything\'s Coming Up Podcast, a Simpsons-focused podcast along with Julia Prescott. Milhouse Mussolini van Houten is a fictional character featured in the animated television series "The Simpsons", voiced by Pamela Hayden, and created by Matt Groening who named the character after President Richard Nixon\'s 

In [23]:
df = {
    'question': ds['question'][:5],
    'answer': ds['answer'][:5],
    'options': ds['options'][:5],
    'original_passages': ds['passages'][:5],
    'new_passages': selected_passages
}

In [27]:
df = pd.DataFrame(df)
df.head()

,question,answer,options,original_passages,new_passages
0,"Which magazine was started first, Arthur's Mag...",Arthur's Magazine,"[""Arthur's Magazine"", 'First for Women']",Arthur's Magazine (1844–1846) was an American ...,Arthur's Magazine (1844–1846) was an American ...
1,Which tennis player won more Grand Slam titles...,Jonathan Stark,"['Henri Leconte', 'Jonathan Stark']",Henri Leconte (born 4 July 1963) is a former F...,Henri Leconte (born 4 July 1963) is a former F...
2,"Which band was founded first, Hole (the rock b...",The Wolfhounds,"['The Wolfhounds', 'Courtney Love']",The Wolfhounds are an indie pop/noise pop band...,The Wolfhounds are an indie pop/noise pop band...
3,Were Pavel Urysohn and Leonid Levin known for ...,no,"['yes', 'no']",Leonid Anatolievich Levin ( ; Russian: Леони́д...,Leonid Anatolievich Levin ( ; Russian: Леони́д...
4,Are both The New Pornographers and Kings of Le...,yes,"['yes', 'no']",Kings of Leon is an American rock band that fo...,Kings of Leon is an American rock band that fo...
